問5<br>
いい機会なので、Gradient Boosting Decision Tree(GBDT)の一種であるXGBoost(eXtremeGradient Boosting)でやってみましょう。<br>
① 実際にやる前に、次の用語を調べてください。
- アンサンブル学習
- ブートストラップ
- バギング
- ブースティング

- アンサンブル学習<br>
精度の低い弱学習器を多く使うことで制度を上げる手法
- ブートストラップ<br>
重複を許して母集団からランダムにデータを流出する手法
- バギング<br>
ブースとトラップを使い抽出した複数のデータ群それぞれで並列的なモデルを構築し学習させる方法
- ブースティング<br>
すべての学習データを使って学習させたモデルを構築し、うまく予測できなかった学習データに対し重みづけをして再学習して新たなモデルを作る、というのを繰り返し、最終的なモデルを作成する

問5<br>
② XGBoostのパラメータについて調べてください。<br>
　 (役割、デフォルト値など)

・eta<br>
-過学習防止のための学習率の調整<br>
-値の範囲:0～1(デフォルト値:0.3)<br>
-小さくすると、モデルの分類性が向上<br>
・min_child_weight<br>
-決定木の葉の重みの下限 -> 下限が大きいほど単調な決定木になるため、過学習抑制<br>
-値の範囲:0～<br>
・max_depth<br>
-決定木の深さの最大値 -> 大きいと過学習しやすくなる<br>
-値の範囲:0～(デフォルト値:6)<br>
・gamma<br>
-決定木の追加による損失減少の下限 <- 設定値だけ損失関数が減少したらノードを分割する<br>
-値の範囲:0～(デフォルト値:0)

問6<br>
(まず)パラメータチューニングなしでXGBoostで予測をし精度検証を
してみてください。<br>
※ XGBoostの使い方は、SVCやランダムフォレストと同じです<br>
　(まずインポートします)->from xgboost import XGBClassifier<br>
　(次に、インスタンスを作ります)->xgb = XGBClassifier()<br>
　あとは'xgb'で.fit()します

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/AIStudy/git/exc_hyperpara_titanic

/content/drive/MyDrive/AIStudy/git/exc_hyperpara_titanic


In [3]:
import pandas as pd

df = pd.read_csv('./data/train.csv')
df = df.drop(['PassengerId','Age','SibSp','Parch','Ticket','Cabin','Name'], axis=1)
df['Embarked'] = df['Embarked'].fillna('S')
df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
df['Sex'] = df['Sex'].apply(lambda x: 0 if x=='male' else 1)
df.head()

,Survived,Pclass,Sex,Fare,Embarked
0,0,3,0,7.2500,0
1,1,1,1,71.2833,1
2,1,3,1,7.9250,0
3,1,1,1,53.1000,0
4,0,3,0,8.0500,0


In [4]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

x = df.drop('Survived', axis=1)
y = df['Survived']
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.3,random_state=0)

xgb = XGBClassifier()
xgb.fit(x_train, y_train)

XGBClassifier()

In [5]:
pred = xgb.predict(x_test)
acc = accuracy_score(pred, y_test)
print('accuracy score : {:.5f}' .format(acc))

accuracy score : 0.80597


問7<br>
XGBoostをグリッドサーチでチューニングしてみてください。<br>
やり方は、ランダムフォレストの時と同じですが、'params'で設定
したパラメータの内容で違うものがあります。<br>
問5で調べたものを
使ってください。

In [6]:
from sklearn.model_selection import GridSearchCV

params = {'eta': [0.01, 0.1, 0.5, 1],
          'min_child_weight': [1, 2, 3, 5, 10],
          'max_depth': [2, 3, 5, 6, 10],
          'gamma': [0, 0.1, 0.2, 0.3, 0.5, 1, 2, 5, 10]
}

xgb = XGBClassifier()
grid = GridSearchCV(xgb, param_grid=params, cv=5, n_jobs=-1)
grid.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=XGBClassifier(), n_jobs=-1,
             param_grid={'eta': [0.01, 0.1, 0.5, 1],
                         'gamma': [0, 0.1, 0.2, 0.3, 0.5, 1, 2, 5, 10],
                         'max_depth': [2, 3, 5, 6, 10],
                         'min_child_weight': [1, 2, 3, 5, 10]})

In [8]:
print('result : {}' .format(grid.cv_results_))
print()
print('best score : {:.5f}' .format(grid.best_score_))
print()
print('best parameters : {}' .format(grid.best_params_))

result : {'mean_fit_time': array([0.04867802, 0.03901563, 0.03548226, 0.03478341, 0.0369947 ,
       0.04100585, 0.041043  , 0.04150424, 0.04089537, 0.04473858,
       0.05502105, 0.05411129, 0.05538387, 0.05357971, 0.04744673,
       0.06509953, 0.05993123, 0.05909767, 0.0567286 , 0.04741893,
       0.08291092, 0.08039942, 0.0795207 , 0.06195779, 0.04757404,
       0.03569326, 0.03468175, 0.03504095, 0.03466501, 0.03644662,
       0.05211897, 0.04042773, 0.04175019, 0.0400291 , 0.04415374,
       0.05444098, 0.05392356, 0.05332637, 0.05333557, 0.04872885,
       0.06082873, 0.06036277, 0.05879846, 0.05745931, 0.0475616 ,
       0.0886003 , 0.08260641, 0.07334161, 0.06317434, 0.04775167,
       0.0346272 , 0.03826361, 0.03620939, 0.03429117, 0.03425293,
       0.04128947, 0.0475111 , 0.04174728, 0.04041338, 0.03990469,
       0.05521936, 0.05874672, 0.05887828, 0.05734224, 0.05210395,
       0.06452312, 0.0609642 , 0.05951867, 0.05815091, 0.0485209 ,
       0.08284845, 0.07709508, 0.07

In [11]:
xgb = XGBClassifier(eta=0.01, gamma=0.2, max_depth=10,min_child_weight=2)
xgb.fit(x_train, y_train)
pred = xgb.predict(x_test)
acc = accuracy_score(pred, y_test)
print('accuracy score : {:.5f}' .format(acc))

accuracy score : 0.80970


In [ ]:
# 解答例では、学習の際はランダムフォレストを上回ってましたが、テストデータでは
# 負けました。テストデータの結果は、'train_test_split'の'random_state'
# の値で変わることがありますが、それが同じ'0'の場合、ランダムフォレストだった
# ということになりますね。

問8<br>
グリッドサーチは、パラメータとして提示した値すべての組合せをしらみつぶしに探索する方法でした。<br>
このやり方はたいへん優れたやり方だとは思いますが、如何せん、組合せが増えると、探索に結構な時間を要する場合があります。<br>
これを解消する方法として、「ランダムサーチ」という方法があります。<br>
ランダムサーチは、しらみつぶしではなく、指定した回数だけランダムな組合せで探索行います。<br>コード的には、グリッドサーチとまったく同じで、GridSearchCVの代わりに、RandomizedSearchCVを使い、パラメータ'n_iter'でランダムサーチの回数を指定します.<br>

タイタニックでランダムサーチを試してみてください。

In [13]:
# 問4と同じ探索範囲設定でランダムフォレストでやってみます
# 問4と同じ探索範囲設定でランダムフォレストでやってみます
import pandas as pd

df = pd.read_csv('./data/train.csv')
df = df.drop(['PassengerId','Age','SibSp','Parch','Ticket','Cabin','Name'], axis=1)

df['Embarked'] = df['Embarked'].fillna('S')
df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
df['Sex'] = df['Sex'].apply(lambda x: 0 if x=='male' else 1)
df.head()

,Survived,Pclass,Sex,Fare,Embarked
0,0,3,0,7.2500,0
1,1,1,1,71.2833,1
2,1,3,1,7.9250,0
3,1,1,1,53.1000,0
4,0,3,0,8.0500,0


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
x = df.drop('Survived', axis=1)
y = df['Survived']
x_train, x_test, y_train, y_test = train_test_split(x, y,
test_size=0.3,
random_state=0)
params = {
'n_estimators': [10, 50, 100, 500, 1000],
'max_depth': [1, 2, 10, 100, 1000],
'max_features': [1, 2, 2.2, 3, 3.5, 4]
}
rs = RandomizedSearchCV(RandomForestClassifier(),
param_distributions=params, cv=5,
n_jobs=-1, n_iter=50)
#しらみつぶしは 5×5×6=150 なので、その 1/3 でやってみます
rs.fit(x_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
60 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py", line 450, in fit
    trees = Parallel(
  File "/usr/local/lib/python3.8/dist-packages/joblib/parallel.py", line 1085, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.8/dist-packages/joblib/parallel.py", 

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'max_depth': [1, 2, 10, 100, 1000],
                                        'max_features': [1, 2, 2.2, 3, 3.5, 4],
                                        'n_estimators': [10, 50, 100, 500,
                                                         1000]})

In [15]:
print('result : {}' .format(rs.cv_results_))
print()
print('best score : {:.5f}' .format(rs.best_score_))
print()
print('best parameters : {}' .format(rs.best_params_))

result : {'mean_fit_time': array([0.27965436, 1.11050653, 0.21172895, 0.54241161, 0.1290072 ,
       0.28543391, 0.23460088, 0.01131372, 0.77542586, 1.14432955,
       0.21383801, 0.02713895, 2.00646091, 0.07306118, 0.45389047,
       0.37090411, 0.19663129, 3.32641473, 0.24736347, 1.08877287,
       2.40372591, 0.04950709, 2.42797012, 0.02575593, 0.10755272,
       0.23427806, 0.22182059, 2.06364279, 0.03130569, 0.12177243,
       0.06489   , 0.12965717, 0.12325711, 0.54881258, 0.11886239,
       1.16090198, 0.10577283, 1.05376282, 0.02926188, 2.43290453,
       2.3605567 , 0.27551012, 2.36356492, 0.11514626, 1.08249454,
       0.24178734, 2.62037935, 0.02730923, 2.85729079, 0.01061916]), 'std_fit_time': array([9.25277073e-03, 9.66629178e-03, 1.12486851e-02, 7.25261628e-03,
       7.39376742e-03, 1.57185225e-02, 5.63778142e-03, 7.92590204e-04,
       6.75061682e-02, 1.26490543e-01, 6.81416728e-03, 6.67291787e-04,
       6.32591328e-01, 8.42171894e-03, 1.56379166e-01, 3.38971070e-02,
 

In [16]:
rf = RandomForestClassifier(max_depth=1000, max_features=4,
n_estimators=500)
rf.fit(x_train, y_train)
pred = rf.predict(x_test)
acc = accuracy_score(pred, y_test)
print('accuracy score : {:.5f}' .format(acc))

accuracy score : 0.81716


In [18]:
# だいたい同精度が出ています。必ずしもうまくいくとは限りませんが、パラメータの組合せが
# ちょっと試しにやってみよう的な場合には有効な方法だと思います。
# もう一つ、有名な方法に「ベイズ最適化」という方法があり、なんでもよさげなパラメータの
# の方向性を探索し、徐々に絞っていくやり方だとか…。私自身、あまり理解できていないので
# 機に紹介させてもらうこととします。